In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO("yolo11n.pt")

In [ ]:
results = model("bus.jpg")

In [ ]:
%matplotlib inline

import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

def visualize_objects(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Run inference
    results = model(image)

    # Convert image to PIL format if it's numpy array
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)

    draw = ImageDraw.Draw(image)

    # Try to load a nice font, fall back to default if not available
    try:
        font = ImageFont.truetype("DejaVuSans.ttf", 20)
    except:
        font = ImageFont.load_default()

    # Generate random colors for each class
    np.random.seed(42)
    colors = np.random.randint(0, 255, size=(len(results[0].names), 3)).tolist()

    # Get the boxes and confidence scores
    boxes = results[0].boxes

    for box in boxes:
        # Get box coordinates
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf[0].cpu().numpy())
        cls = int(box.cls[0].cpu().numpy())

        # Skip if confidence is below threshold
        if conf < 0.3:
            continue

        # Draw bounding box
        color = tuple(colors[cls])
        draw.rectangle([x1, y1, x2, y2], outline=color, width=3)

        # Add label with class name and confidence
        label = f"{results[0].names[cls]}: {conf:.2f}"
        label_size = draw.textbbox((0, 0), label, font=font)[2:]

        # Draw label background
        draw.rectangle([x1, y1 - label_size[1], x1 + label_size[0], y1],
                    fill=color)

        # Draw label text
        draw.text((x1, y1 - label_size[1]), label, fill=(255, 255, 255),
                font=font)from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
from PIL import Image
import io

app = FastAPI(title="YOLO Object Detection API")

# Configure CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In production, replace with specific origins
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/detect")
async def detect_objects(file: UploadFile = File(...)):
    """
    Endpoint to perform object detection on uploaded images
    """
    try:
        # Verify that model is loaded
        if model is None:
            return JSONResponse(
                status_code=500,
                content={"error": "Model not properly loaded"}
            )
            
        # Read and process the image
        contents = await file.read()
        image = Image.open(io.BytesIO(contents))
        
        # Perform detection
        results = model(image)
        
        # Process results
        detections = []
        for result in results:
            boxes = result.boxes
            for box in boxes:
                # Get box coordinates
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                
                # Get confidence score
                confidence = float(box.conf[0])
                
                # Get class name
                class_id = int(box.cls[0])
                class_name = result.names[class_id]
                
                detections.append({
                    "bbox": [x1, y1, x2, y2],
                    "confidence": confidence,
                    "class": class_name
                })
        
        return {
            "success": True,
            "detections": detections,
            "message": f"Processed {file.filename} successfully",
            "num_detections": len(detections)
        }
        
    except Exception as e:
        return JSONResponse(
            status_code=500,
            content={
                "success": False,
                "error": str(e)
            }
        )

@app.get("/health")
async def health_check():
    """
    Endpoint to check if the server is running and model is loaded
    """
    return {
        "status": "healthy",
        "model_loaded": model is not None
    }

if __name__ == "__main__":
    # Run the server
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=8000,
        reload=True  # Set to False in production
    )

    # Display results
    plt.figure(figsize=(15, 10))
    plt.imshow(image)
    plt.axis('off')

    return results

In [ ]:
results = visualize_objects("bus.jpg")

# Print detection summary
print("\nDetection Summary:")
print("-----------------")
for box in results[0].boxes:
    conf = float(box.conf[0].cpu().numpy())
    cls = int(box.cls[0].cpu().numpy())
    if conf >= 0.3:
        print(f"Class: {results[0].names[cls]}, Confidence: {conf:.2f}")
